### Present Cipher (80-bit)
Implemented for CYBERUS Project  
Biplab Gautam  

Note: This implementation is not optimized and is done only to understand the Present algorithm.

In [2]:
import numpy as np

sBox = [0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0XD, 0X3, 0XE, 0XF, 0X8, 0X4, 0X7, 0X1, 0X2]
permutation = [0,16,32,48,1,17,33,49,2,18,34,50,3,19,35,51,4,
               20,36,52,5,21,37,53,6,22,38,54,7,23,39,55,8,24,
               40,56,9,25,41,57,10,26,42,58,11,27,43,59,12,28,
               44,60,13,29,45,61,14,30,46,62,15,31,47,63]

def key_to_matrix(hex_key):
    matrix = []
    for i in range(20):
        byte = (hex_key >> (4 * (19 - i))) & 0xF
        matrix.append(byte)
    return matrix

def matrix_to_key(matrix_key):
    key_value = 0x0
    for i in range(len(matrix_key)):
        key_value = (matrix_key[i]) ^ key_value<<4
        
    return key_value

def generate_round_keys(master_key):
    # Master key 80-bit = 0x12a0b8a898b98098c090
    matrix_key = key_to_matrix(master_key)
    round_keys = [matrix_to_key(matrix_key[:16])]
    for i in range(1, 32):
        # Rotate the key by 61 positions to the left
        key_value = matrix_to_key(matrix_key) 
        key_value = (key_value << 61 | key_value >> (80-61)) & 0xffffffffffffffffffff
        matrix_key = key_to_matrix(key_value)
        # Pass the leftmost four bits to S-box
        matrix_key[0] = sBox[matrix_key[0]]
        # XOR the k19tok15 bits with round counter i. This affects two blocks. 4th and 5th block.
        matrix_key[16] = matrix_key[16] ^ ((i&0b1)<<3)
        matrix_key[15] = matrix_key[15] ^ ((i>>1)&0b1111)
        # Append the matrix to round_keys
        round_keys.append(matrix_to_key(matrix_key[:16]))
    return round_keys

# master_key = 0x12a0b8a898b98098c090
# master_key = 0x00000000000000000000
# generate_round_keys(master_key)

In [3]:
# Encryption

def data_to_matrix(hex_data):
    matrix = []
    for i in range(16):
        byte = (hex_data >> (4 * (15 - i))) & 0xF
        matrix.append(byte)
    return matrix

def matrix_to_data(matrix_data):
    key_value = 0x0
    for i in range(len(matrix_data)):
        key_value = (matrix_data[i]) ^ key_value<<4
        
    return key_value

def addRoundKey(key, data):
    """
    Key is 64-bit hex
    Data is 64-bit hex
    """
    return key ^ data

def sBoxLayer(data):
    """
    data is 64-bit hex
    """
    matrix_data = data_to_matrix(data)
    for i, nibble in  enumerate(matrix_data):
        matrix_data[i] = sBox[nibble]
    return matrix_to_data(matrix_data)

def pLayer(data):
    """
    data is 64-bit hex
    """
    original_data = list(format(data, '#066b')[2:])
    permuted_data = list(format(data, '#066b')[2:])
    for i, j in enumerate(permutation):
        permuted_data[j] = original_data[i]
    return int("0b" + "".join(permuted_data), 2)


In [4]:
def encrypt(master_key, data):
    """
    master_key: 64-bit hex
    data: 64-bit hex
    """
    round_keys = generate_round_keys(master_key)
    for i in range(31):
        print(f"\nRound {i}: \n")
        data = addRoundKey(round_keys[i], data)
        print("After round_key addition:", hex(data))
        data = sBoxLayer(data)
        print("After sbox layer:", hex(data))
        data = pLayer(data)
        print("Round output: ",hex(data))
        print("--------------------------")
    data = addRoundKey(round_keys[31], data)
    
    return data

data = 0x0
key = 0x0
encrypt(key, data)


Round 0: 

After round_key addition: 0x0
After sbox layer: 0xcccccccccccccccc
Round output:  0xffffffff00000000
--------------------------

Round 1: 

After round_key addition: 0x3fffffff00000000
After sbox layer: 0xb2222222cccccccc
Round output:  0x80ff00ffff008000
--------------------------

Round 2: 

After round_key addition: 0xd0ff18ffff008001
After sbox layer: 0x7c22532222cc3cc5
Round output:  0x4036c837b7c88c09
--------------------------

Round 3: 

After round_key addition: 0x2036c237b4c88c08
After sbox layer: 0x6cba46bd894334c3
Round output:  0x73c2cd26b6192359
--------------------------

Round 4: 

After round_key addition: 0xc3c2c126b759233b
After sbox layer: 0x4b46456a8d0e6bb8
Round output:  0x41d7be58531e4446
--------------------------

Round 5: 

After round_key addition: 0xd1d7a858529e446c
After sbox layer: 0x757df30306e199a4
Round output:  0x182ef861ad62fd1c
--------------------------

Round 6: 

After round_key addition: 0x182f6a61afa2fd2f
After sbox layer: 0x5362afa5

6159166413734052933

In [5]:
hex(6159166413734052933)

'0x5579c1387b228445'

Test vectors to compare the results can be obtained here:  
https://crypto.stackexchange.com/questions/26037/test-vectors-for-present-algorithm  
https://www.youtube.com/watch?v=tLa0IBpOE_I&t=629s  